In [1]:
"""
1. Programmatic download of TLE data
2. Compute satellite positions over time
3. Calculate look angles from a specified ground station
4. 2D plots showing ground tracks over Earth’s map
5. Dynamic plot of azimuth vs. time
6. Description of orbital mechanics and propagation algorithms (in comments)
7. Output screenshots and graphs from simulations (to be generated by functions)
8. Summary of results (to be written in notebook markdown)
9. End-to-end demonstration in Jupyter Notebook
"""
pass

In [27]:
!pip install --quiet sgp4 plotly ipywidgets pyproj

import numpy as np
import matplotlib.pyplot as plt
from sgp4.api import Satrec, WGS72, jday
from datetime import datetime, timedelta
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyproj import Transformer
from google.colab import files
import io
import plotly.io as pio
pio.renderers.default = "colab"


In [29]:
def load_tle_file_from_io(uploaded_io):
    lines = uploaded_io.strip().splitlines()
    tle_records = []
    for i in range(0, len(lines), 3):
        try:
            tle_records.append((lines[i].strip(), lines[i+1].strip(), lines[i+2].strip()))
        except IndexError:
            continue
    return tle_records



In [30]:
def propagate_orbit(line1: str, line2: str, start_dt: datetime, end_dt: datetime, step_s: int = 60):
    sat = Satrec.twoline2rv(line1, line2, WGS72)
    dt = timedelta(seconds=step_s)
    times, positions = [], []
    t = start_dt
    while t <= end_dt:
        y, mo, d, hr, mi, sec = t.timetuple()[:6]
        jd, fr = jday(y, mo, d, hr, mi, sec + t.microsecond * 1e-6)
        err, r, v = sat.sgp4(jd, fr)
        positions.append(np.array(r) if err == 0 else None)
        times.append(t)
        t += dt
    return times, positions

In [31]:
def eci_to_geodetic(r_eci, dt_utc):
    gmst = (18.697374558 + 24.06570982441908 * ((dt_utc - datetime(2000, 1, 1)).total_seconds() / 86400.0)) % 24
    theta = np.deg2rad(gmst * 15)
    R = np.array([[np.cos(theta), np.sin(theta), 0],
                  [-np.sin(theta), np.cos(theta), 0],
                  [0, 0, 1]])
    r_ecef = R.dot(r_eci)
    transformer = Transformer.from_crs(
        {"proj": "geocent", "ellps": "WGS84", "datum": "WGS84"},
        "EPSG:4326",
        always_xy=True
    )
    lon, lat, alt = transformer.transform(r_ecef[0]*1000, r_ecef[1]*1000, r_ecef[2]*1000)
    return lat, lon, alt / 1000



In [32]:
def calculate_look_angles(r_ecis, gs_lat, gs_lon, gs_alt):
    transformer = Transformer.from_crs(
        "EPSG:4326",
        {"proj": "geocent", "ellps": "WGS84", "datum": "WGS84"},
        always_xy=True
    )
    xg, yg, zg = transformer.transform(gs_lon, gs_lat, gs_alt * 1000)
    vecs = [(r * 1000 - np.array([xg, yg, zg])) for r in r_ecis]
    rngs = [np.linalg.norm(v) / 1000 for v in vecs]
    azs = [np.degrees(np.arctan2(v[1], v[0])) % 360 for v in vecs]
    els = [np.degrees(np.arcsin(v[2] / np.linalg.norm(v))) for v in vecs]
    return azs, els, rngs


In [33]:
def plot_azimuth_time(times, azimuths):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=times, y=azimuths, mode='lines+markers', name='Azimuth'))
    fig.update_layout(title='Azimuth vs. Time', xaxis_title='Time (UTC)', yaxis_title='Azimuth (deg)')
    fig.show()

In [34]:
def plot_ground_track_with_time(times, lats, lons):
    fig = go.Figure()
    fig.add_trace(go.Scattergeo(
        lon = lons,
        lat = lats,
        text = [t.strftime('%H:%M:%S') for t in times],
        mode = 'markers+text',
        marker = dict(size=4),
        textposition='top center'
    ))
    fig.update_layout(
        title = 'Ground Track with Time Labels',
        geo = dict(
            projection_type = 'equirectangular',
            showland = True,
            landcolor = "rgb(212, 212, 212)",
        )
    )
    fig.show()

In [35]:
sat_dropdown = widgets.Dropdown(description='Satellite:')
lat_input = widgets.FloatText(value=40.0, description='GS Lat (°)')
lon_input = widgets.FloatText(value=-105.0, description='GS Lon (°)')
alt_input = widgets.FloatText(value=1.6, description='GS Alt (km)')
duration_input = widgets.FloatText(value=2.0, description='Dur (hrs)')
run_btn = widgets.Button(description='Run Simulation')
output = widgets.Output()

In [36]:
fetch_records = []

def on_upload(change):
    with output:
        clear_output()
        try:
            upload_obj = next(iter(upload_btn.value.values()))
            content = upload_obj['content']
            if isinstance(content, bytes):
                text = io.StringIO(content.decode("utf-8"))
            else:
                text = io.StringIO(content)
            global fetch_records
            fetch_records = load_tle_file_from_io(text.read())
            sat_dropdown.options = [name for name, _, _ in fetch_records]
            sat_dropdown.value = sat_dropdown.options[0]
            print(f"Loaded {len(fetch_records)} TLE entries from {upload_obj['metadata']['name']}.")
        except Exception as e:
            print("Failed to load TLE file:", e)

upload_btn = widgets.FileUpload(accept='.tle,.txt', multiple=False)
upload_btn.observe(on_upload, names='value')

def on_run(_):
    with output:
        clear_output()
        try:
            name = sat_dropdown.value
            for rec in fetch_records:
                if rec[0] == name:
                    name, l1, l2 = rec
                    break
            else:
                raise ValueError("Selected satellite not found in TLE records.")

            print(f"Simulating: {name}")
            start = datetime.utcnow()
            end = start + timedelta(hours=duration_input.value)
            times, pos = propagate_orbit(l1, l2, start, end)
            valid = [(t, r) for t, r in zip(times, pos) if r is not None]
            if not valid:
                raise ValueError("All satellite positions failed to propagate. Check TLE validity.")
            times, pos = zip(*valid)
            lats, lons, alts = zip(*[eci_to_geodetic(r, t) for r, t in zip(pos, times)])
            az, el, rng = calculate_look_angles(pos, lat_input.value, lon_input.value, alt_input.value)
            plot_ground_track_with_time(times, lats, lons)
            plot_azimuth_time(times, az)
        except Exception as e:
            print(e)

run_btn.on_click(on_run)

ui = widgets.VBox([
    upload_btn,
    sat_dropdown,
    widgets.HBox([lat_input, lon_input, alt_input, duration_input]),
    run_btn,
    output
])
display(ui)
